***GENERATED CODE FOR predictivechurndatapipeline PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '559', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phone', 'transformation_label': 'String Indexer'}], 'feature': 'Phone', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '559', 'mean': '', 'stddev': '', 'min': '327-3954', 'max': '422-8344', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phone'}, {'feature_label': 'Phone', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phone')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'Intl_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '559', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intl_Plan'}, {'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intl_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'VMail_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '559', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'VMail_Plan'}, {'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('VMail_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'}], 'feature': 'cluster_labels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '559', 'mean': '', 'stddev': '', 'min': 'day_callers', 'max': 'vmailers', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'cluster_labels'}, {'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('cluster_labels')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run predictivechurndatapipelineHooks.ipynb
try:
	#sourcePreExecutionHook()

	predictivechurntrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/TrainData/PredictiveChurnTrain.csv', 'filename': 'PredictiveChurnTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(predictivechurntrain)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run predictivechurndatapipelineHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(predictivechurntrain,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "559", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Account_Length", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "103.0", "stddev": "39.77", "min": "1", "max": "232", "missing": "0"}, "updatedLabel": "Account_Length"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Area_Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "437.01", "stddev": "42.32", "min": "408", "max": "510", "missing": "0"}, "updatedLabel": "Area_Code"}, {"transformationsData": [{"feature_label": "Phone", "transformation_label": "String Indexer"}], "feature": "Phone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "559", "mean": "", "stddev": "", "min": "327-3954", "max": "422-8344", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phone"}, {"transformationsData": [{"feature_label": "Intl_Plan", "transformation_label": "String Indexer"}], "feature": "Intl_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "559", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intl_Plan"}, {"transformationsData": [{"feature_label": "VMail_Plan", "transformation_label": "String Indexer"}], "feature": "VMail_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "559", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "VMail_Plan"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "VMail_Message", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "7.87", "stddev": "13.63", "min": "0", "max": "46", "missing": "0"}, "updatedLabel": "VMail_Message"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "178.29", "stddev": "54.02", "min": "19.0", "max": "335.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "99.48", "stddev": "21.11", "min": "42", "max": "163", "missing": "0"}, "updatedLabel": "Day_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "29.88", "stddev": "9.19", "min": "3.0", "max": "57.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "202.11", "stddev": "49.94", "min": "42.0", "max": "354.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "99.55", "stddev": "20.81", "min": "36", "max": "164", "missing": "0"}, "updatedLabel": "Eve_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "16.71", "stddev": "4.26", "min": "3.0", "max": "30.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "200.47", "stddev": "47.76", "min": "57.0", "max": "352.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "99.69", "stddev": "19.99", "min": "33", "max": "157", "missing": "0"}, "updatedLabel": "Night_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "8.55", "stddev": "2.17", "min": "2.0", "max": "15.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "9.85", "stddev": "2.89", "min": "0.0", "max": "20.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "592.07", "stddev": "87.98", "min": "284.0", "max": "882.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "4.44", "stddev": "2.44", "min": "0", "max": "15", "missing": "0"}, "updatedLabel": "Intl_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "2.3", "stddev": "0.81", "min": "0.0", "max": "5.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "559", "mean": "58.93", "stddev": "10.26", "min": "25.0", "max": "90.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustServ_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "1.6", "stddev": "1.38", "min": "0", "max": "8", "missing": "0"}, "updatedLabel": "CustServ_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Churn", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "559", "mean": "0.12", "stddev": "0.32", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Churn"}, {"transformationsData": [{"feature_label": "cluster_labels", "transformation_label": "String Indexer"}], "feature": "cluster_labels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "559", "mean": "", "stddev": "", "min": "day_callers", "max": "vmailers", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "cluster_labels"}, {"feature": "State_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "559", "mean": "21.03", "stddev": "14.56", "min": "0.0", "max": "50.0", "missing": "0"}, "updatedLabel": "State_stringindexer_trans..."}, {"feature": "Phone_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "559", "mean": "279.0", "stddev": "161.51", "min": "0.0", "max": "558.0", "missing": "0"}, "updatedLabel": "Phone_stringindexer_trans..."}, {"feature": "Intl_Plan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "559", "mean": "0.08", "stddev": "0.27", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Intl_Plan_stringindexer_t..."}, {"feature": "VMail_Plan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "559", "mean": "0.27", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "VMail_Plan_stringindexer_..."}, {"feature": "cluster_labels_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "559", "mean": "2.39", "stddev": "1.71", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "cluster_labels_stringinde..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run predictivechurndatapipelineHooks.ipynb
try:
	#sinkPreExecutionHook()

	extra = HDFSConnector.put(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/extra/PredictiveChurnTrain_DataCatalog.csv', 'pathOnly': '/FileStore/platform/extra', 'filename': 'PredictiveChurnTrain_DataCatalog', 'token': '', 'delimiter': ','}")
	#sinkPostExecutionHook(extra)

except Exception as ex: 
	logging.error(ex)
